In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
import apiCalls.polygon as polygon
import apiCalls.alpha as alpha
import database as database
import sqlite3

import seaborn as sns
import pandas as pd
import numpy as np



In [16]:
database.create_database()
json_polygon = polygon.get_data('VOO')
json_alpha = alpha.get_data('VOO')

In [21]:
data_list = alpha.json_to_list(json_alpha)
response = database.save_data_to_database(data_list)
print(data_list)
print(type(data_list[0][2]))



[['VOO', datetime.date(2024, 6, 4), 484.0, 486.41, 482.56, 485.74, 2546556], ['VOO', datetime.date(2024, 6, 3), 486.3, 486.41, 480.4, 485.15, 4211256], ['VOO', datetime.date(2024, 5, 31), 481.24, 484.91, 476.475, 484.62, 5333971], ['VOO', datetime.date(2024, 5, 30), 482.12, 482.7504, 479.2, 480.44, 3393589], ['VOO', datetime.date(2024, 5, 29), 483.19, 484.68, 482.96, 483.69, 5604961], ['VOO', datetime.date(2024, 5, 28), 487.4, 487.645, 484.525, 487.12, 4034716], ['VOO', datetime.date(2024, 5, 24), 485.18, 487.4, 484.33, 486.73, 2555241], ['VOO', datetime.date(2024, 5, 23), 489.91, 489.99, 482.29, 483.44, 3811307], ['VOO', datetime.date(2024, 5, 22), 487.79, 488.43, 484.94, 487.06, 3250581], ['VOO', datetime.date(2024, 5, 21), 486.53, 488.58, 486.35, 488.48, 2453576], ['VOO', datetime.date(2024, 5, 20), 486.78, 488.6099, 486.45, 487.17, 2599585], ['VOO', datetime.date(2024, 5, 17), 486.09, 486.73, 484.7, 486.69, 3274035], ['VOO', datetime.date(2024, 5, 16), 487.05, 488.57, 485.83, 485.9

In [3]:

class SymbolDataset(Dataset):
  def __init__(self, symbol, transform=None):
    self.symbol = symbol
    self.conn = sqlite3.connect('historical.db')
    self.cursor = self.conn.cursor()
    self.cursor.execute(f'SELECT * FROM stock_data WHERE symbol="{self.symbol}"')
    self.data = self.cursor.fetchall()
    self.conn.close()

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    return self.data[idx]
  
  @property
  def columns(self):
    return ['date', 'open', 'high', 'low', 'close', 'volume']
  

In [4]:
spxDataset = SymbolDataset('VOO')
